# Deutsch-Jozsa Algorithm

In this notebook I will prove the quantum advantage for the parity problem. 
## Imports and requirements


To run this notebook it is requiered a python versin >= 3.10, personally, I am using python 3.11

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
# Requieremts
%pip install os
%pip install qiskit
%pip install qiskit_ibm_runtime
%pip install numpy
%pip install matplot
%pip ipywidgets
%pip install python-dotenv


In [15]:
# Imports
from dotenv import load_dotenv
import os
from qiskit_ibm_runtime import QiskitRuntimeService


## IBM Quantum Runtime setup
Firstly we will setup the qiskit_ibm_runtime service so we can execute our circuit on a real hardware.

In [ ]:
# Cargar automáticamente el archivo .env de la raíz del proyecto
load_dotenv(override=True)

# Comprobar que las variables se han cargado
print("🔑 Token (primeros 6 chars):", os.environ["IBM_QUANTUM_TOKEN"][:6], "...")

# Configurar el servicio de IBM Quantum
service = QiskitRuntimeService(
    channel=os.environ["IBM_QUANTUM_CHANNEL"],   # "ibm_cloud"
    token=os.environ["IBM_QUANTUM_TOKEN"],
    instance=os.environ["IBM_QUANTUM_INSTANCE"]  # CRN
)


# Deutsch-Jozsa-algorithm-implementation

## Introduction
The Deutsch–Jozsa algorithm distinguishes between constant and balanced functions exponentially faster than any classical deterministic algorithm. Even if it is not a usefull algorithm in a practical way, it is a simple example to show the quantum advantage.


## Algorithm description
Deutsch algorithm solves the parity problem for functions such us:

$$
\sum \;\; \longrightarrow \sum
$$
The outputs will be:

$$  0 \Leftrightarrow \; constant $$
$$ 1 \Leftrightarrow \; balanced$$

<br>
Deutsch-Jozsa algorithm is the extension of the Deustch algorithm for functions such us:

$$\sum^n \;\;\longrightarrow\;\; \sum \; , \quad n \geq 1 $$

To describe how this algorithm works , imagine this game: Alice, in LA sends a bit string to Bob who is in NY. Bob calculates the image of some function $ f(x) $ that could work in two different ways. First  $f(x) = 0 \ $  or  $ f(x) = 1 $ ,but never both, constant. Otherwise $ f(x) = 0 $ half of the times , $ f(x) = 1 $ the other half, balanced. The goal of Alice is knowing how this $ f(x) $ work, constant or balanced. 

If Bob agrees to use an unitary operator $ U_f $ then, Alice only needs a single query to know how the function $ f(x) $ behaves. In contrast the best classical method would require $ 2^{n-1} +1 $  queries.

## Implementation

## Referances 
Nielsen & Chuang, secciones 1.4.2–1.4.4.





ttps://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa


## To-do

AÑADIR REFERENCIA A PRIMERA PARTE "ALGORITHM DESCRIPTION" ALGO COMO  SEE ...

We will follow the qiskit patern